In [27]:
import pandas as pd

In [28]:
data = pd.read_excel("C:\\Users\\rubya\\Downloads\\championleaguedata10.xlsx")

In [29]:
data.head()

,name,position,match played,Minutes played,Goals,Ballls recovered(defending),assist(attacking),Passing accuracy (%),Top speed (km/h),Distance covered (km),Yellow cards,Red cards,Team
0,STEFAN ORTEGA,Goalkeeper,2,180,0,0,0,0.9600,23.60,10.77,0,0,Man city
1,kyle walker,defender,4,272,0,22,0,0.6467,35.21,29.89,0,0,Man city
2,Ruben dias,defender,11,958,1,59,1,0.9564,33.00,120.59,2,0,Man city
3,John stones,defender,7,603,1,26,1,0.9286,33.80,72.91,0,0,Man city
4,Nathen ake,defender,7,518,0,33,0,0.9543,31.50,59.93,1,0,Man city


In [30]:
data.describe()

,match played,Minutes played,Goals,Ballls recovered(defending),assist(attacking),Passing accuracy (%),Top speed (km/h),Distance covered (km),Yellow cards,Red cards
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,8.142857,532.833333,1.166667,20.880952,0.976190,0.820455,31.944524,64.420952,0.928571,0.023810
std,3.375460,318.017174,2.070898,18.085437,1.489608,0.104526,2.252096,38.708986,0.947213,0.154303
min,1.000000,14.000000,0.000000,0.000000,0.000000,0.500000,23.600000,2.630000,0.000000,0.000000
25%,5.250000,277.000000,0.000000,6.250000,0.000000,0.753075,31.050000,31.032500,0.000000,0.000000
50%,9.000000,527.000000,0.500000,14.500000,0.000000,0.831500,32.350000,62.095000,1.000000,0.000000
75%,11.000000,826.500000,1.000000,33.000000,2.000000,0.909125,33.000000,97.297500,1.000000,0.000000
max,12.000000,1080.000000,12.000000,65.000000,7.000000,0.960000,35.800000,123.360000,4.000000,1.000000


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   name                         42 non-null     object 
 1   position                     42 non-null     object 
 2   match played                 42 non-null     int64  
 3   Minutes played               42 non-null     int64  
 4   Goals                        42 non-null     int64  
 5   Ballls recovered(defending)  42 non-null     int64  
 6   assist(attacking)            42 non-null     int64  
 7   Passing accuracy (%)         42 non-null     float64
 8   Top speed (km/h)             42 non-null     float64
 9   Distance covered (km)        42 non-null     float64
 10  Yellow cards                 42 non-null     int64  
 11  Red cards                    42 non-null     int64  
 12  Team                         42 non-null     object 
dtypes: float64(3), int64(7

In [33]:
#EDA AND FEATURE ENGINEERING


# Perform feature engineering
data['TotalCards'] = data['Yellow cards'] + data['Red cards']

# Scaling the 'Minutes played' feature to a range of 0-1
data['MinutesScaled'] = (data['Minutes played'] - data['Minutes played'].min()) / (data['Minutes played'].max() - data['Minutes played'].min())

# Binning the 'Goals' feature into categories (e.g., 'Low', 'Medium', 'High')
bins = [0, 2, 5, data['Goals'].max()]
labels = ['Low', 'Medium', 'High']
data['GoalsCategory'] = pd.cut(data['Goals'], bins=bins, labels=labels)

#dropping the extra column
data = data.drop('name', axis=1)

# One-hot encoding for categorical variables (e.g., 'position' and 'Team')
data_encoded = pd.get_dummies(data, columns=['position', 'Team'])

# Print the updated data with engineered features
print(data_encoded.head())



   match played  Minutes played  Goals  Ballls recovered(defending)  \
0             2             180      0                            0   
1             4             272      0                           22   
2            11             958      1                           59   
3             7             603      1                           26   
4             7             518      0                           33   

   assist(attacking)  Passing accuracy (%)  Top speed (km/h)  \
0                  0                0.9600             23.60   
1                  0                0.6467             35.21   
2                  1                0.9564             33.00   
3                  1                0.9286             33.80   
4                  0                0.9543             31.50   

   Distance covered (km)  Yellow cards  Red cards  TotalCards  MinutesScaled  \
0                  10.77             0          0           0       0.155722   
1                  29.89    

In [34]:


# Define the feature engineering functions
def calculate_goal_to_minute_ratio(row):
    if row['Minutes played'] != 0:
        return row['Goals'] / row['Minutes played']
    else:
        return 0

#feature engineering functions to create new features
data['Goal-to-minute ratio'] = data.apply(calculate_goal_to_minute_ratio, axis=1)

# Select the relevant columns including the new features as features (X) and the target variable (y)
features = ['match played', 'Minutes played', 'Goals', 'Ballls recovered(defending)', 'assist(attacking)',
            'Passing accuracy (%)', 'Top speed (km/h)', 'Distance covered (km)', 'Yellow cards', 'Red cards',
            'Goal-to-minute ratio']
target = 'Team'

X = data[features]
y = data[target]



In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Select the relevant columns as features (X) and the target variable (y)
features = ['match played', 'Minutes played', 'Goals', 'Ballls recovered(defending)', 'assist(attacking)', 'Passing accuracy (%)', 'Top speed (km/h)', 'Distance covered (km)', 'Yellow cards', 'Red cards']
target = 'Team'

X = data[features]
y = data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with feature scaling and logistic regression
pipeline = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', max_iter=1000))

# Define the hyperparameter grid for tuning
param_grid = {'logisticregression__C': [0.1, 1, 10], 'logisticregression__penalty': ['l1', 'l2']}

# Perform grid search cross-validation to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Make predictions on the test data
predictions = best_model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)



Accuracy: 0.7777777777777778


In [ ]:
# so our model is 77.8% accurate.
